In [ ]:
#| default_exp dr

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| export

import subprocess
def get_gpu_memory(device = 0):
    total_memory = subprocess.check_output(["nvidia-smi", "--query-gpu=memory.total", "--format=csv,noheader,nounits", "--id=" + str(device)])
    total_memory = int(total_memory.decode().split('\n')[0])
    used_memory = subprocess.check_output(["nvidia-smi", "--query-gpu=memory.used", "--format=csv,noheader,nounits",  "--id=" + str(device)])
    used_memory = int(used_memory.decode().split('\n')[0])

    percentage = round((used_memory / total_memory) * 100)
    return used_memory, total_memory, percentage

def color_for_percentage(percentage):
    if percentage < 20:
        return "\033[90m"  # Gray
    elif percentage < 40:
        return "\033[94m"  # Blue
    elif percentage < 60:
        return "\033[92m"  # Green
    elif percentage < 80:
        return "\033[93m"  # Orange
    else:
        return "\033[91m"  # Red
        
def create_bar(percentage, color_code, length=20):
    filled_length = int(length * percentage // 100)
    bar = "█" * filled_length + "-" * (length - filled_length)
    return color_code + bar + "\033[0m"  # Apply color and reset after bar

def gpu_memory_status(device=0):
    used, total, percentage = get_gpu_memory(device)
    color_code = color_for_percentage(percentage)
    bar = create_bar(percentage, color_code)
    print(f"Used mem: {used}")
    print(f"Used mem: {total}")
    print(f"Memory Usage: [{bar}] {color_code}{percentage}%\033[0m")

# Dimensionality reduction

In [ ]:
#| export
import umap
import cudf
import cuml
import pandas as pd
import numpy as np
from fastcore.all import *
from dvats.imports import *
from dvats.load import TSArtifact

In [ ]:
#| export
def check_compatibility(dr_ar:TSArtifact, enc_ar:TSArtifact):
    "Function to check that the artifact used by the encoder model and the artifact that is \
    going to be passed through the DR are compatible"
    try:
        # Check that both artifacts have the same variables
        chk_vars = dr_ar.metadata['TS']['vars'] == enc_ar.metadata['TS']['vars']
        # Check that both artifacts have the same freq
        chk_freq = dr_ar.metadata['TS']['freq'] == enc_ar.metadata['TS']['freq']
        # Check that the dr artifact is not normalized (not normalized data has not the key normalization)
        chk_norm = dr_ar.metadata['TS'].get('normalization') is None
        # Check that the dr artifact has not missing values
        chk_miss = dr_ar.metadata['TS']['has_missing_values'] == "False"
        # Check all logical vars.
        if chk_vars and chk_freq and chk_norm and chk_miss:
            print("Artifacts are compatible.")
        else:
            raise Exception
    except Exception as e:
        print("Artifacts are not compatible.")
        raise e
    return None

## Get projections (UMAP, T-SNET, PCA)

In [ ]:
#| export
#Comment this part after 4_seconds debugged
import hashlib

In [ ]:
#| export
import warnings
import sys
from numba.core.errors import NumbaPerformanceWarning
@delegates(cuml.UMAP)
def get_UMAP_prjs(
    input_data, 
    cpu=True, 
    print_flag = False, 
    check_memory_usage = True,
    **kwargs
):
    "Compute the projections of `input_data` using UMAP, with a configuration contained in `**kwargs`."
    if print_flag: 
        print("--> get_UMAP_prjs")
        print("kwargs: ", kwargs)
        sys.stdout.flush()
        ####
        checksum = hashlib.md5(input_data.tobytes()).hexdigest()
        print(checksum)
        ####
        
    if check_memory_usage: gpu_memory_status()
    
    warnings.filterwarnings("ignore", category=NumbaPerformanceWarning) # silence NumbaPerformanceWarning
    
    #reducer = umap.UMAP(**kwargs) if cpu else cuml.UMAP(**kwargs)
    if cpu:
        print("-- umap.UMAP --", cpu)
        sys.stdout.flush()
        reducer = umap.UMAP(**kwargs)
    else:
        print("-- cuml.UMAP --", cpu)
        sys.stdout.flush()
        if 'random_state' in kwargs:
            kwargs['random_state'] = np.uint64(kwargs['random_state'])
        reducer = cuml.UMAP(**kwargs)
    
    if print_flag: 
        print("------- reducer --------")
        print(reducer)
        print(reducer.get_params())
        print("------- reducer --------")
        sys.stdout.flush()
    
    projections = reducer.fit_transform(input_data)
    
    if check_memory_usage: gpu_memory_status()
    if print_flag: 
        checksum = hashlib.md5(projections.tobytes()).hexdigest()
        print("prjs checksum ", checksum)
        print("get_UMAP_prjs -->")
        sys.stdout.flush()
    return projections

In [ ]:
#| slow
foo = np.random.rand(5, 10)
bar = get_UMAP_prjs(
    foo, 
    cpu=False, 
    print_flag = True,
    check_memory_usage = True,
    random_state = 1234, #822569775
    n_neighbors=3, 
    min_dist=0.1
)
test_eq(bar.shape, (foo.shape[0], 2))

--> get_UMAP_prjs
kwargs:  {'random_state': 1234, 'n_neighbors': 3, 'min_dist': 0.1}
1511f9e3c0ce668bd54ce23099714673
Used mem: 2454
Used mem: 24576
Memory Usage: [██------------------] 10%
1234
RandomState(MT19937)
-- cuml.UMAP -- False
------- reducer --------
UMAP()
{'handle': <pylibraft.common.handle.Handle object>, 'verbose': 4, 'output_type': 'input', 'n_neighbors': 3, 'n_components': 2, 'n_epochs': None, 'learning_rate': 1.0, 'min_dist': 0.1, 'spread': 1.0, 'set_op_mix_ratio': 1.0, 'local_connectivity': 1.0, 'repulsion_strength': 1.0, 'negative_sample_rate': 5, 'transform_queue_size': 4.0, 'init': 'spectral', 'a': 1.5769434601962196, 'b': 0.8950608779914887, 'target_n_neighbors': -1, 'target_weight': 0.5, 'target_metric': 'categorical', 'hash_input': False, 'random_state': 822569775, 'callback': None, 'metric': 'euclidean', 'metric_kwds': None, 'precomputed_knn': None}
------- reducer --------
Used mem: 2462
Used mem: 24576
Memory Usage: [██------------------] 10%
prjs checksum 

In [ ]:
#| slow
foo = np.random.rand(5, 10)
bar = get_UMAP_prjs(
    foo, 
    cpu=True, 
    print_flag = True,
    check_memory_usage = True
    n_neighbors=3, 
    min_dist=0.1
)
test_eq(bar.shape, (foo.shape[0], 2))

--> get_UMAP_prjs
kwargs:  {'n_neighbors': 3, 'min_dist': 0.1}
175a6ecb96ef4de7bcbf326ae624d879
Used mem: 2428
Used mem: 24576
Memory Usage: [██------------------] 10%
-- cuml.UMAP -- False
------- reducer --------
UMAP()
{'handle': <pylibraft.common.handle.Handle object>, 'verbose': 4, 'output_type': 'input', 'n_neighbors': 3, 'n_components': 2, 'n_epochs': None, 'learning_rate': 1.0, 'min_dist': 0.1, 'spread': 1.0, 'set_op_mix_ratio': 1.0, 'local_connectivity': 1.0, 'repulsion_strength': 1.0, 'negative_sample_rate': 5, 'transform_queue_size': 4.0, 'init': 'spectral', 'a': 1.5769434601962196, 'b': 0.8950608779914887, 'target_n_neighbors': -1, 'target_weight': 0.5, 'target_metric': 'categorical', 'hash_input': False, 'random_state': 2997075094, 'callback': None, 'metric': 'euclidean', 'metric_kwds': None, 'precomputed_knn': None}
------- reducer --------
Used mem: 2438
Used mem: 24576
Memory Usage: [██------------------] 10%
prjs checksum  28f08a32beaf2872fd87d57b643a166d
get_UMAP_prjs

If you want to have consistent results across executions, use `random_state`

In [ ]:
#| hide
bar = get_UMAP_prjs(foo, cpu=True, n_neighbors=3, random_state=1234)
baz = get_UMAP_prjs(foo, cpu=True, n_neighbors=3, random_state=1234)
test_eq(bar, baz)

Used mem: 1944
Used mem: 24576
Memory Usage: [█-------------------] 8%
-- umap.UMAP -- True


/home/macu/env/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


Used mem: 1944
Used mem: 24576
Memory Usage: [█-------------------] 8%
Used mem: 1944
Used mem: 24576
Memory Usage: [█-------------------] 8%
-- umap.UMAP -- True


/home/macu/env/lib/python3.10/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


Used mem: 1944
Used mem: 24576
Memory Usage: [█-------------------] 8%


In [ ]:
#| export
@delegates(cuml.PCA)
def get_PCA_prjs(X, cpu=False, **kwargs):
    r"""
    Computes PCA projections of X
    """
    if cpu:
        raise NotImplementedError
    else:
        reducer = cuml.PCA(**kwargs)
    projections = reducer.fit_transform(X)
    return projections

In [ ]:
#| hide
# Test the function get_PCA_prjs
foo = np.random.rand(5, 10)
bar = get_PCA_prjs(foo, cpu=False, n_components=2)

In [ ]:
#| export
@delegates(cuml.TSNE)
def get_TSNE_prjs(X, cpu=False, **kwargs):
    r"""
    Computes TSNE projections of X
    """
    if cpu:
        raise NotImplementedError
    else:
        reducer = cuml.TSNE(**kwargs)
    projections = reducer.fit_transform(X)
    return projections

In [ ]:
#| Test the function get_TSNE_prjs
foo = np.random.rand(90, 10)
bar = get_TSNE_prjs(foo, cpu=False)

/home/macu/env/lib/python3.10/site-packages/cuml/internals/api_decorators.py:342: UserWarning: Starting from version 22.04, the default method of TSNE is 'fft'.
  return func(**kwargs)


## Export 

In [ ]:
#| export 

from sklearn.metrics import silhouette_score
def cluster_score(prjs, clusters_labels, print_flag):
    score = silhouette_score(prjs, clusters_labels)
    if print_flag: print("Silhouette_score:", score)
    return score

In [ ]:
#| hide
#from nbdev.export import notebook2script
#notebook2script()
beep(1)